#  使用 Tensorflow 和 Pytorch 创建模型

在迄今为止的教程中，我们已经使用了 DeepChem 提供的标准模型。这对许多应用来说是很好的，但你迟早会想用你自己定义的架构创建一个全新的模型。DeepChem 提供了与 TensorFlow（Keras）和 PyTorch的集成，所以你可以使用这些框架中的任何一个模型。

## Colab

本教程和目录中的其余部分都是在 Google colab 中完成。如果您想在 colab 中打开此笔记本，您可以点击以下链接。

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/BioGavin/deepchem/blob/master/examples/tutorials-zh/Creating_Models_with_TensorFlow_and_PyTorch.ipynb)



In [ ]:
!pip install --pre deepchem

实际上，你可以采取两种不同的方法来使用 TensorFlow 或 PyTorch 模型与 DeepChem 结合，这取决于你是否想使用 TensorFlow/PyTorch APIs 或 DeepChem APIs 来训练和评估你的模型。对于前一种情况，DeepChem 的 `Dataset` 类有一些方法，可以轻松地调整它以用于其他框架。`make_tf_dataset()` 返回一个 `tensorflow.data.Dataset` 数据迭代对象。`make_pytorch_dataset()` 返回一个`torch.utils.data.IterableDataset` 对数据进行迭代。这让你可以使用 DeepChem 的数据集、加载器、特征生成器、转化器、分割器等，并轻松地将它们集成到你现有的 TensorFlow 或 PyTorch 代码中。

但是 DeepChem 也提供了许多其他有用的功能，另一种方法是将你的模型包裹在 DeepChem 的 `Model` 对象中，让我们来看看如何做到这一点。

## KerasModel

`KerasModel` 是 DeepChem 的 `Model` 类的一个子类，它充当了 `tensorflow.keras.Model` 的封装器。让我们看一个使用它的例子，在这个例子中，我们创建了一个由两个稠密层组成的简单顺序模型。

In [1]:
import deepchem as dc
import tensorflow as tf

keras_model = tf.keras.Sequential([
    tf.keras.layers.Dense(1000, activation='relu'),
    tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.Dense(1)
])
model = dc.models.KerasModel(keras_model, dc.models.losses.L2Loss())

在这个例子中，我们使用了 Keras 的 `Sequential` 类。我们的模型包括一个带有 ReLU 激活的密集层，50% 的 dropout 以提供正则化，以及一个产生标量输出的最终层。我们还需要指定训练模型时使用的损失函数，在本例中为 L<sub>2</sub> Loss。我们现在可以像使用任何其他 DeepChem 模型一样训练和评估模型。例如，让我们加载 Delaney 溶解度数据集。我们的模型该如何根据分子的扩展连接性指纹 (ECFP)预测分子的溶解度呢？

In [2]:
tasks, datasets, transformers = dc.molnet.load_delaney(featurizer='ECFP', splitter='random')
train_dataset, valid_dataset, test_dataset = datasets
model.fit(train_dataset, nb_epoch=50)
metric = dc.metrics.Metric(dc.metrics.pearson_r2_score)
print('training set score:', model.evaluate(train_dataset, [metric]))
print('test set score:', model.evaluate(test_dataset, [metric]))

training set score: {'pearson_r2_score': 0.9787445597470444}
test set score: {'pearson_r2_score': 0.736905850092889}


## TorchModel

`TorchModel` 的工作方式与 `KerasModel` 类似，只是它包装了 `torch.nn.Module`。让我们使用 PyTorch 创建另一个模型，和之前的模型构造一样，并在相同的数据上对其进行训练。

In [3]:
import torch

pytorch_model = torch.nn.Sequential(
    torch.nn.Linear(1024, 1000),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.5),
    torch.nn.Linear(1000, 1)
)
model = dc.models.TorchModel(pytorch_model, dc.models.losses.L2Loss())

model.fit(train_dataset, nb_epoch=50)
print('training set score:', model.evaluate(train_dataset, [metric]))
print('test set score:', model.evaluate(test_dataset, [metric]))

training set score: {'pearson_r2_score': 0.9798256761766225}
test set score: {'pearson_r2_score': 0.7256745385608444}


## 计算损失

现在让我们看一个更高级的例子。在上述模型中，损失是直接从模型的输出中计算出来的。这通常很好，但并非总是如此。考虑一个输出概率分布的分类模型，虽然可以从概率中计算出损失，但从对数中计算出损失在数值上更稳定。

为了做到这一点，我们创建一个模型，返回多个输出，包括概率和对数。`KerasModel` 和 `TorchModel` 允许你指定一个 "output types" 的列表。如果一个特定的输出有 "predict" 的类型，这意味着它是一个正常的输出，当你调用 `predict()` 时应该被返回。如果它的类型是 "loss"，这意味着它应该被传递给损失函数，以代替正常输出。

顺序模型不允许有多个输出，所以我们创建一个子类模型。

In [4]:
class ClassificationModel(tf.keras.Model):
    
    def __init__(self):
        super(ClassificationModel, self).__init__()
        self.dense1 = tf.keras.layers.Dense(1000, activation='relu')
        self.dense2 = tf.keras.layers.Dense(1)

    def call(self, inputs, training=False):
        y = self.dense1(inputs)
        if training:
            y = tf.nn.dropout(y, 0.5)
        logits = self.dense2(y)
        output = tf.nn.sigmoid(logits)
        return output, logits

keras_model = ClassificationModel()
output_types = ['prediction', 'loss']
model = dc.models.KerasModel(keras_model, dc.models.losses.SigmoidCrossEntropy(), output_types=output_types)

我们可以在 BACE 数据集上训练我们的模型。这是一个二元分类任务，试图预测一个分子是否会抑制 BACE-1 酶。

In [5]:
tasks, datasets, transformers = dc.molnet.load_bace_classification(feturizer='ECFP', split='scaffold')
train_dataset, valid_dataset, test_dataset = datasets
model.fit(train_dataset, nb_epoch=100)
metric = dc.metrics.Metric(dc.metrics.roc_auc_score)
print('training set score:', model.evaluate(train_dataset, [metric]))
print('test set score:', model.evaluate(test_dataset, [metric]))

training set score: {'roc_auc_score': 0.9996116504854369}
test set score: {'roc_auc_score': 0.7701992753623188}


## Other Features

`KerasModel`和`TorchModel`有很多其他的功能，以下是一些比较重要的功能。

- 在训练中自动保存检查点；
- 将进度记录到 [TensorBoard](https://www.tensorflow.org/tensorboard) 或者 [Weights & Biases](https://docs.wandb.com/) 控制台；
- 用 `f(output, labels, weights)` 形式的函数自定义损失函数；
- 使用 `ValidationCallback` 类提前停止；
- 从预训练的模型中加载参数；
- 估计模型输出的不确定性；
- 通过显著性图谱识别重要特征。

通过在 "KerasModel"或 "TorchModel" 中包装你自己的模型，你可以立即获得所有这些功能。 关于它们的全部细节，请参见API文档。

# 恭喜！是时候加入社区了！

恭喜您完成本教程笔记本！如果您喜欢本教程并希望继续使用 DeepChem，我们鼓励您完成本系列的其余教程。您还可以通过以下方式帮助 DeepChem 社区：

## 在 [GitHub](https://github.com/deepchem/deepchem) 上为 DeepChem 点亮小星星
这有助于大家建立对 DeepChem 项目和我们正在尝试构建的开源药物发现工具的共识。

## 加入 DeepChem Gitter
DeepChem [Gitter](https://gitter.im/deepchem/Lobby) 聚集了许多对生命科学深度学习感兴趣的科学家、开发人员和爱好者，欢迎加入！